In [1]:
# Importing required libraries

import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from pymongo import MongoClient

### Connection to MongoDB and simulation

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador']

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [4]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\JCristancho\AppData\Local\Temp\ipykernel_11892\1910320475.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
# this function is created to simulate the opinion editorial category
def make_request1(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
        
    #simulate button click    
    for i in range (5):
        element = browser.find_element(by=By.CSS_SELECTOR, value='div[class*="pure-button"]')
        browser.execute_script("arguments[0].click();", element)
        
    time.sleep(40)
    
    browser.execute_script('window.scrollTo(0,-250);')

   
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')


### Extract and organize the "El Espectador" information

In [8]:
def upload_news(category):
    
    news = []
    
    for i in range(2):
        # Getting HTML content for news listing page
        soup = make_request(browser, '/archivo/'+category+'/'+str((i+1))+'/')
        
        # Finding the section where news are contained 
        layout = soup.find(class_ = 'Layout-flexAds')
        # Getting blocks from layout
        blocks = layout.find('section').find_all(recursive = False)
        
        cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
              
        # Building a list with title and relative path of the news founded
        for card in cards:
            news.append({
                'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
                'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
            })
            
            
    return news



In [9]:
def order_information(news):
    for n in news:
        # Getting HTML content for news page
        soup = make_request(browser, n['relative_path'])
        
        # conditional that determines if there is a date or not
        if soup.find(class_ = 'ArticleHeader-Date')  is not None:
            # Extracting news metadata
            n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
         
        else:
            
            n['datetime'] = 'No date'
        
        
        # conditional that determines if the article contains an author or not
        if soup.find(class_ = 'ACredit-Author').find('a') is not None:
            
            # Extracting news metadata
            n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text()
        
        else:
            
            n['author'] = 'no author'
            
           
        # conditional that determines if there is a summary or not
        if soup.find(class_ = 'ArticleHeader-Hook') is not None:
            
            # Extracting news metadata
            n['summary'] =soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
                
        else:  
            
            n['summary'] ='No summary'
            
        
        soup1 = soup
        # remove <a> tag from html
        [s.extract() for s in soup1("a")]
        
        # Extracting and concatenating news full text
        paragraphs = soup1.find_all(class_ = 'font--secondary')
        n['full_text'] = ' '.join([p.get_text() for p in paragraphs])
                
    return news


### Web scraping political category

In [10]:
news = upload_news('politica')
len(news)

12

In [11]:
# Storing extracted information for further analysis
collection.insert_many(order_information(news))

###  Web scraping category editorial opinion

In [12]:
news = []

# Getting HTML content for news listing page
soup = make_request1(browser, '/opinion/editorial/')

# Finding the section where news are contained 
layout = soup.find(class_ = 'Layout-flexAds')
# Getting blocks from layout
blocks = layout.find('section').find_all(recursive = False)

# Finding and concatenating news cards
cards =  blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
            
# Building a list with title and relative path of the news founded
for card in cards:
    news.append({
        'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
        'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
    })

len(news)

5

In [13]:
# Storing extracted information for further analysis
collection.insert_many(order_information(news))

###  Web scraping category Bogotá

In [14]:
news = upload_news('bogota')
len(news)

12

In [15]:
# Storing extracted information for further analysis
collection.insert_many(order_information(news))

###  Web scraping category Colombia

In [16]:
news = upload_news('colombia')
len(news)

12

In [17]:
# Storing extracted information for further analysis
collection.insert_many(order_information(news))

### Web scraping category Judicial

In [18]:
news = upload_news('judicial')
len(news)

12

In [19]:
# Storing extracted information for further analysis
collection.insert_many(order_information(news))